# Poem RNN

Generating a poem based on Shakespeare.

In [78]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Embedding
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils

import sys

tf.random.set_seed(1)

In [79]:
from google.colab import drive
drive.mount('/content/drive')
drive_dir = '/content/drive/My Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Data

In [80]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

Now we need to make a way to transer from characters to numbers and numbers to characters so two dictionarys are made.

In [81]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

Now we are going to take a sequence of 100 of the characters and save the label of the next character. This essentially makes our data for x and y.

In [82]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(length-seq_length):
     sequence = text[i:i + seq_length]
     label =text[i + seq_length]
     X.append([char_to_n[char] for char in sequence])
     Y.append(char_to_n[label])

Simple preprocessing for our model.

In [83]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

LSTM model.

In [84]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [87]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=100)

11153/11153 [==============================] - 681s 61ms/step - loss: 2.3166


In [88]:
model.save_weights(drive_dir + '/saved_models/text_generator_400_0.2_400_0.2_shakespeare.h5')

In [89]:
model.load_weights(drive_dir + '/saved_models/text_generator_400_0.2_400_0.2_shakespeare.h5')

Now we can print the predictions. Below a starter string is given and then the model makes predictions using softmax and the np.argmax finds the position of the highest predictions. It then appends that corresponding letter to the string from the beginning.

In [92]:
string_mapped = X[199]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [93]:
txt=""
for char in full_string:
    txt = txt+char
print(txt)

u know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let uhe searee of the world the will the searent
That the world the will the will the will the searen of the searent
That the world the will the will the baute the will the searent
That the world the will the will the baute the will the searent That she would the will the will the baute the will the searen of the searent That she would the will the baute the will the baute the searent That she would th
